# Быстро строим деревья


Деревья глубины 1 обычно называют решающими пнями. 
Как Вы думаете, какая сложность построения решающего пня?


Представим, что мы решаем задачу регрессии, считаем для одного сплита. 

$$Q = F(R_m) - \frac{N_l}{N_m} F(R_l) - \frac{N_r}{N_m} F(R_r) $$

$$F(R) = \frac{1}{N} \sum_{i=1}^{N}(mean(y) - y_i) ^ 2 $$

$$Q = F(R_m) - \frac{1}{N_m} [ \sum_{i \in R_l}(mean(y_l) - y_i) ^ 2  +  \sum_{i \in R_r}(mean(y_r) - y_i) ^ 2 ]  $$

$F(R_m) и N_m$ от сплита не зависит, можно отбросить. Осталось научиться быстро считать такие выражения:
$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2$ для всех порогов. 

Если делать в лоб, то получится $O(||R||^2)$, нам не нравится, очень долго. Каждый раз пересчитывать довольно глупо, потому что мы точно знаем, какой именно объект при изменении сплита перешел из левой части в правую!




Воспользуемся формулой для дисперсии:
$$D(R) = mean(y ^ 2) -  mean(y) ^ 2 =   \frac{1}{N} \sum_{i \in R} y_i^2 -  \frac{1}{N ^ 2} (\sum_{i \in R} y_i) ^2$$

Но у нас не дисперсия, а дисперсия без деления на число объектов, то есть:
$$\sum_{i \in R_l}(mean(y_l) - y_i) ^ 2 =   \sum_{i \in R_k} y_i^2 -  \frac{1}{N_l} (\sum_{i \in R_l} y_i) ^2 $$
Аналогично для правого сплита. 

Вот мы и получили более простой алгоритм подсчета сплита:

1) Отсортировали фичу и таргет по фиче

2) Сначала считаем, что порог это минимальное значение, то есть левый кусок пустой, правый все значения. Храним для каждой из частей
а) Сумму
б) Сумму квадратов
в) Число объектов

3) Взяли следующее значение в отсортированном списке, это значение ушло от правого куска в левое. Обновили а) б) в) в обоих кусках

4) Посчитали  для каждого куска б) - а) ** 2 / в), сложили для обоих кусков. Если эта сумма маеньше лучшего значения, то это лучший сплит из текущих


Реализуйте алгоритм, сравните качество со sklearn.

Если получилось одинаково, Вы молодец!!!

In [1]:
import numpy as np

In [72]:
def get_threshold(feature, y):
    best_thr = None
    best_score = None
    sq_sum_l = np.dot(y, y)
    sum_y_l = np.sum(y)
    sum_sq_l = sum_y_l**2
    sq_sum_r = 0
    sum_y_r = 0
    sum_sq_r = 0
    index = np.argsort(feature)
    feature = feature[index]
    y = y[index]
    for i, (f, y_i) in enumerate(zip(feature, y)):
        if best_thr is None:
            best_score = sq_sum_l - 1/(len(y) - i) * sum_sq_l
            best_thr = f
        else:
            tmp_score = sq_sum_l - 1/(len(y) - i) * sum_sq_l + sq_sum_r - 1/(i) * sum_sq_r
            if tmp_score < best_score:
                best_thr = f
                best_score = tmp_score
                
        sq_sum_l -= y_i**2
        sum_y_l -= y_i
        sum_sq_l -= y_i*(2 * sum_y_l + y_i)
        sq_sum_r += y_i**2
        sum_y_r += y_i
        sum_sq_r += y_i*(2 * sum_y_r + y_i)
        
    return best_score, best_thr


def train_stump(X, y):
    best_f = None
    best_thr = None
    best_error = float('inf')
    left_mean, right_mean = None, None
    for j in range(X.shape[1]):
        f_score, f_thr = get_threshold(X[:, j], y)
        if f_score < best_error:
            best_q = f_score
            best_f = j
            best_thr = f_thr
    maskl = X[:, best_f] < best_thr
    maskr = X[:, best_f] >= best_thr
    left_mean = np.mean(y[maskl]) if len(y[maskl]) != 0 else 0
    right_mean = np.mean(y[maskr]) if len(y[maskr]) != 0 else 0
    return best_f, best_thr, left_mean, right_mean


    

In [29]:
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)

In [77]:
%%time
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
best_f, best_thr, left_mean, right_mean = train_stump(X, y)
maskl = X[:, best_f] < best_thr
maskr = X[:, best_f] >= best_thr
pred = np.zeros(len(y))
pred[maskl] = left_mean
pred[maskr] = right_mean
print(mean_squared_error(y, pred))
# regressor_stump = DecisionTreeRegressor(random_state=0, max_depth=1).fit(X, y)
# print(mean_squared_error(y, regressor_stump.predict(X)))

48.08665360896714
CPU times: user 35.6 ms, sys: 500 µs, total: 36.1 ms
Wall time: 34.8 ms


In [30]:
%%time
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
regressor = DecisionTreeRegressor(random_state=0).fit(X, y)
print(mean_squared_error(y, regressor.predict(X)))
regressor_stump = DecisionTreeRegressor(random_state=0, max_depth=1).fit(X, y)
print(mean_squared_error(y, regressor_stump.predict(X)))

0.0
46.19909167710848
CPU times: user 7.91 ms, sys: 3.12 ms, total: 11 ms
Wall time: 9.86 ms
